In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import zipfile
import io
import unicodedata
from concurrent.futures import ThreadPoolExecutor
import streamlit as st

# --- 1. RECUPERAR CONTEXTO DE LA APP PRINCIPAL ---
PALETA = globals().get("active_palette", ["#0576F3", "#36F48C", "#F47806"])
FUENTE = globals().get("active_font", "Arial")
BUSQUEDA_RAW = globals().get("LOCALIDAD_SELECCIONADA", "")

if not BUSQUEDA_RAW:
    try:
        BUSQUEDA_RAW = input("")
    except:
        BUSQUEDA_RAW = "Monterrey"

# --- 2. FUNCIONES DE UTILIDAD (Con Caché de Streamlit) ---

def normalizar_texto(texto):
    if not isinstance(texto, str):
        return str(texto)
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode("utf-8")
    return texto.lower().strip()

@st.cache_data(show_spinner=False, ttl=3600)
def descargar_datos_inegi():
    fuentes = [
        {"anio": 2005, "url": "https://www.inegi.org.mx/contenidos/programas/ccpv/2005/datosabiertos/cpv2005_iter_00_csv.zip", "csv_pattern": "cpv2005_iter_00.csv", "encoding": "latin-1"},
        {"anio": 2010, "url": "https://www.inegi.org.mx/contenidos/programas/ccpv/2010/datosabiertos/iter_nal_2010_csv.zip", "csv_pattern": "iter_00_cpv2010.csv", "encoding": "latin-1"},
        {"anio": 2020, "url": "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/iter/iter_00_cpv2020_csv.zip", "csv_pattern": "conjunto_de_datos_iter_00CSV20.csv", "encoding": "utf-8"}
    ]

    def _proceso_individual(fuente):
        try:
            response = requests.get(fuente['url'], timeout=30)
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                nombre_archivo = next((f for f in z.namelist() if fuente['csv_pattern'] in f), None)
                if nombre_archivo:
                    with z.open(nombre_archivo) as csv_file:
                        df = pd.read_csv(csv_file, encoding=fuente['encoding'], dtype=str)
                        col_loc = "NOM_LOC" if "NOM_LOC" in df.columns else "nom_loc"
                        cols_interes = [col_loc, "p_total", "p_mas", "p_fem", "pobtot", "pobmas", "pobfem", "POBTOT", "POBMAS", "POBFEM"]
                        cols_existentes = [c for c in cols_interes if c in df.columns]
                        df = df[cols_existentes].copy()
                        df["nom_loc_norm"] = df[col_loc].apply(normalizar_texto)
                        return fuente['anio'], df
        except Exception as e:
            return fuente['anio'], None

    dataframes = {}
    with ThreadPoolExecutor(max_workers=3) as executor:
        resultados = executor.map(_proceso_individual, fuentes)
        for anio, df in resultados:
            if df is not None:
                dataframes[anio] = df

    return dataframes

# --- 3. LÓGICA PRINCIPAL ---

if not BUSQUEDA_RAW:
    st.warning("👆 Por favor ingresa un nombre de localidad para buscar.")
else:
    termino_busqueda = normalizar_texto(BUSQUEDA_RAW)

    with st.spinner(f"📥 Consultando bases de datos históricas del INEGI para '{BUSQUEDA_RAW}'..."):
        dataframes = descargar_datos_inegi()

    df05, df10, df20 = dataframes.get(2005), dataframes.get(2010), dataframes.get(2020)

    if df05 is None or df10 is None or df20 is None:
        st.error("❌ Error de conexión con INEGI. Intenta más tarde.")
    else:
        coincidencias = df20[df20["nom_loc_norm"].str.contains(termino_busqueda)]
        lista_localidades = coincidencias[["nom_loc_norm", "NOM_LOC"]].drop_duplicates(subset="nom_loc_norm")

        if lista_localidades.empty:
            st.warning(f"⚠ No se encontraron localidades que contengan: '{BUSQUEDA_RAW}'")
        else:
            st.success(f"✅ Se encontraron {len(lista_localidades)} localidades.")

            def extraer_datos(row_df, cols_map):
                if row_df.empty:
                    return {"total": 0, "h": 0, "m": 0}
                row = row_df.iloc[0].to_dict()
                res = {}
                for key_out, key_in in cols_map.items():
                    try:
                        res[key_out] = float(row.get(key_in, 0))
                    except:
                        res[key_out] = 0
                return res

            # --- GRAFICAR CADA COINCIDENCIA ---
            for idx, item in lista_localidades.iterrows():
                nombre_norm = item["nom_loc_norm"]
                nombre_real = item["NOM_LOC"]

                r05 = df05[df05["nom_loc_norm"] == nombre_norm]
                r10 = df10[df10["nom_loc_norm"] == nombre_norm]
                r20 = df20[df20["nom_loc_norm"] == nombre_norm]

                d05 = extraer_datos(r05, {"total": "p_total", "h": "p_mas", "m": "p_fem"})
                d10 = extraer_datos(r10, {"total": "pobtot", "h": "pobmas", "m": "pobfem"})
                d20 = extraer_datos(r20, {"total": "POBTOT", "h": "POBMAS", "m": "POBFEM"})

                anios = ["2005", "2010", "2020"]
                poblacion_total = [d05["total"], d10["total"], d20["total"]]
                hombres = [d05["h"], d10["h"], d20["h"]]
                mujeres = [d05["m"], d10["m"], d20["m"]]

                c_principal = PALETA[0] if len(PALETA) > 0 else "#0b132b"
                c_secundario = PALETA[1] if len(PALETA) > 1 else "#ff9f18"
                c_terciario = PALETA[2] if len(PALETA) > 2 else "#cccccc"

                fig = make_subplots(rows=1, cols=2, subplot_titles=(
                    "Crecimiento Total",
                    "Distribución por Género"
                ))

                fig.add_trace(go.Bar(
                    x=anios, y=poblacion_total,
                    text=[f"{int(v):,}" for v in poblacion_total],
                    textposition="auto", marker_color=c_principal, name="Total"
                ), row=1, col=1)

                fig.add_trace(go.Bar(
                    x=anios, y=hombres,
                    text=[f"{int(v):,}" for v in hombres],
                    textposition="auto", marker_color=c_secundario, name="Hombres"
                ), row=1, col=2)

                fig.add_trace(go.Bar(
                    x=anios, y=mujeres,
                    text=[f"{int(v):,}" for v in mujeres],
                    textposition="auto", marker_color=c_terciario, name="Mujeres"
                ), row=1, col=2)

                fig.update_layout(
                    title=dict(
                        text=f"📍 {nombre_real}",
                        x=0.5,
                        xanchor='center'
                    ),
                    title_font=dict(size=20, family=FUENTE),
                    font=dict(family=FUENTE),
                    barmode="group",
                    height=500,
                    margin=dict(t=80, b=100, l=40, r=40),
                    plot_bgcolor="rgba(0,0,0,0)",
                    paper_bgcolor="rgba(0,0,0,0)",
                    showlegend=True,
                    legend=dict(
                        orientation="h",
                        yanchor="top",
                        y=-0.15,
                        xanchor="center",
                        x=0.5
                    ),
                    annotations=[
                        dict(
                            x=0,
                            y=-0.28,
                            xref="paper",
                            yref="paper",
                            text="Fuente: Censos de Población y Vivienda (2005, 2010, 2020), INEGI.",
                            showarrow=False,
                            font=dict(size=12, color="gray"),
                            xanchor="left",
                            yanchor="top"
                        )
                    ]
                )

                st.plotly_chart(fig, use_container_width=True)

                # ---------------------------------------------------------
                # TABLA DE DATOS (Visible, sin expander)
                # ---------------------------------------------------------
                st.markdown("**Datos detallados**")

                # Creamos el DataFrame con los datos recopilados
                df_tabla = pd.DataFrame({
                    "Año": anios,
                    "Población Total": poblacion_total,
                    "Hombres": hombres,
                    "Mujeres": mujeres
                })

                # Ordenamos descendente para mostrar lo más reciente primero
                df_tabla = df_tabla.sort_values("Año", ascending=False)

                st.dataframe(
                    df_tabla,
                    use_container_width=True,
                    hide_index=True,
                    column_config={
                        "Año": st.column_config.TextColumn("Año"), # Texto para evitar comas (2,020)
                        "Población Total": st.column_config.NumberColumn(format="%,d"),
                        "Hombres": st.column_config.NumberColumn(format="%,d"),
                        "Mujeres": st.column_config.NumberColumn(format="%,d"),
                    }
                )

                st.markdown("---")


⬇️  Iniciando descarga y procesamiento...
   ⏳ Descargando 2005...
   ⏳ Descargando 2010...
   ⏳ Descargando 2020...
   ✅ 2005 listo.
   ✅ 2010 listo.
   ✅ 2020 listo.

Ingresa parte del nombre de la localidad (ej. 'regla'): regla

🔍 Buscando todas las coincidencias para 'regla'...
✅ Se encontraron 5 localidades. Generando gráficas...

   📊 Graficando: Puente Regla


   📊 Graficando: San Miguel Regla


   📊 Graficando: Santa María Regla


   📊 Graficando: El Regladero


   📊 Graficando: El Regladero (Los Tanques)



✅ Proceso finalizado.
